In [1]:
import pandas as pd
import random
import sys
sys.path.append('../..')

from model import run
from model import plot_utils as pu
from model import processing
from model.amm.omnipool_amm import OmnipoolState
from model.amm.agents import Agent
from model.amm.trade_strategies import random_swaps
from model.amm.global_state import GlobalState

# same seed, same parameters = same simulation result
random.seed(42)

price_dict = {
    'HDX': 0.05,
    'USD': 1,
    'LRNA': 0.5,
    'R1': 2,
    'R2': 2/3
}

def lrna_price(asset: str):
    return price_dict['LRNA'] / price_dict[asset]

initial_state = GlobalState(
    pools={
        'omnipool': OmnipoolState(
            tokens={
                'HDX': {'liquidity': 1000000, 'LRNA_price': lrna_price('HDX')},
                'USD': {'liquidity': 1000000, 'LRNA_price': lrna_price('USD')},
                'R1': {'liquidity': 1000000, 'LRNA_price': lrna_price('R1')},
                'R2': {'liquidity': 3000000, 'LRNA_price': lrna_price('R2')},
            },
            lrna_fee=0,
            asset_fee=0
        )
    },
    agents = {
        'Trader': Agent(
            holdings={'HDX': 10000, 'USD': 1000000, 'R1': 1000000, 'R2': 3000000},
            trade_strategy=random_swaps(
                pool_id='omnipool', 
                amount={
                    'R1': 100 * lrna_price('R1'), 
                    'R2': 100  * lrna_price('R2'), 
                    'HDX': 100 * lrna_price('HDX'), 
                    'USD': 100 * lrna_price('USD')
                }
            )
        )
    }
)

events = run.run(initial_state, time_steps=5000)

events = processing.postprocessing(events)

SyntaxError: invalid syntax (trade_strategies.py, line 262)

In [ ]:
pu.plot(events, pool='omnipool', prop='liquidity')
pu.plot(events, pool='omnipool', prop='lrna')

In [ ]:
pu.plot(events, agent='Trader', prop='holdings')